# Classificação de Imagens de Arco senil

In [1]:
from keras.layers import Activation,Convolution2D, Dropout,Conv2D,Dense
from keras.layers import AveragePooling2D,BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras.callbacks import ReduceLROnPlateau
from keras import layers
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import scipy.misc
from imutils import paths
import cv2
import os
import random

Using TensorFlow backend.


In [2]:
# pq esses módulos?
# pq essa divisão?
def model_cnn_2(classes):
    model = Sequential()
    model.add(Convolution2D(filters=16,kernel_size=(7,7),activation='selu',padding='same',
                           name='image_array',input_shape=(150,150,1)))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=16,kernel_size=(7,7),activation='elu',
                           strides=(2,2),padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=48,kernel_size=(5,5),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Convolution2D(filters=48,kernel_size=(5,5),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    #model.add(Activation('relu'))
    
    model.add(Convolution2D(filters=192,kernel_size=(3,3),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=192,kernel_size=(3,3),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=128,kernel_size=(3,3),activation='selu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(4096,activation='selu'))
    model.add(Dropout(0.5))
    model.add(layers.Dense(4096,activation='selu'))
    model.add(Dropout(0.5))
    model.add(layers.Dense(classes,activation='sigmoid'))
    
    return model
    

In [3]:
def model_cnn_vgg16(classes):
    #224 224
    _input = Input((150,150,1)) 

    conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
    conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
    pool1  = MaxPooling2D((2, 2))(conv2)

    conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
    conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
    pool2  = MaxPooling2D((2, 2))(conv4)

    conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
    conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
    conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
    pool3  = MaxPooling2D((2, 2))(conv7)

    conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
    conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
    conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
    pool4  = MaxPooling2D((2, 2))(conv10)

    conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
    conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
    conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
    pool5  = MaxPooling2D((2, 2))(conv13)

    flat   = Flatten()(pool5)
    dense1 = Dense(4096, activation="relu")(flat)
    dense2 = Dense(4096, activation="relu")(dense1)
    output = Dense(classes, activation="sigmoid")(dense2)

    vgg16_model  = Model(inputs=_input, outputs=output)
    
    return vgg16_model

In [4]:
model = model_cnn_2(1)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 150, 150, 16)      800       
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        12560     
_________________________________________________________________
batch_normalization_2 (Batch (None, 75, 75, 16)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 48)        19248     
_________________________________________________________________
batch_normalization_3 (Batch (None, 37, 37, 48)        192       
__________

In [6]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

carregando imagens 

In [9]:
print("Carregando imagens...")
data = []
labels = []

imagePaths = sorted(list(paths.list_images('/home/jailsonpereira/PAIC_CNN/versao_1')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath,0)
    image = cv2.resize(image,(150,150))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
data = np.array(data,dtype='float32')/255
labels = np.array(labels)

print("Imagens carregadas")
print(data.shape)
print(labels.shape)

Carregando imagens...
Imagens carregadas
(223, 150, 150)
(223,)


In [10]:
#data = data.reshape(-1,224,224,1)
#labels = labels.reshape(-1,224,224,1)

In [11]:
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)

In [12]:
labels.shape

(223,)

In [13]:
data.shape

(223, 150, 150)

particionando e normalizando dataset

In [14]:
(trainX,testX,trainY,testY) = train_test_split(data,labels,test_size=0.1,random_state=42)

In [15]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(200, 150, 150)
(200,)
(23, 150, 150)
(23,)


In [16]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
#X_train = trainX.re
trainX = trainX.reshape(-1,150,150,1)
testX = testX.reshape(-1,150,150,1)

In [17]:
#conveter o labels para inteiros em um vetor (para 2-class, binary)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [18]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(200, 150, 150, 1)
(200, 1)
(23, 150, 150, 1)
(23, 1)


In [19]:
optimizer = optimizers.rmsprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [20]:
# Data argumentation
aug = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.1, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [21]:
aug.fit(trainX)

In [22]:
EPOCHS = 150
BS = 32

In [23]:
model.compile(loss="binary_crossentropy", optimizer=optimizer,
        metrics=["accuracy"])

In [24]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [25]:
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=trainX.shape[0]/10,
    epochs=EPOCHS,callbacks=[learning_rate_reduction])

Epoch 1/150
20/20 [==============================] - 5s 265ms/step - loss: 4.2094 - acc: 0.7120 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 2/150
20/20 [==============================] - 2s 124ms/step - loss: 4.9599 - acc: 0.6889 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 3/150
20/20 [==============================] - 2s 117ms/step - loss: 4.9259 - acc: 0.6910 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 4/150
20/20 [==============================] - 2s 117ms/step - loss: 4.1645 - acc: 0.7388 - val_loss: 2.7726 - val_acc: 0.8261

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.000500000023749.
Epoch 5/150
20/20 [==============================] - 2s 118ms/step - loss: 4.5620 - acc: 0.7138 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 6/150
20/20 [==============================] - 2s 118ms/step - loss: 4.4658 - acc: 0.7199 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 7/150
20/20 [==============================] - 2s 115ms/step - loss: 4.5079 - acc: 0.7172 - val_loss: 2.7726 - val_acc:

20/20 [==============================] - 2s 116ms/step - loss: 4.7424 - acc: 0.7025 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 120/150
20/20 [==============================] - 2s 112ms/step - loss: 4.4367 - acc: 0.7217 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 121/150
20/20 [==============================] - 2s 111ms/step - loss: 4.5580 - acc: 0.7141 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 122/150
20/20 [==============================] - 2s 112ms/step - loss: 4.6332 - acc: 0.7094 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 123/150
20/20 [==============================] - 2s 120ms/step - loss: 4.5569 - acc: 0.7142 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 124/150
20/20 [==============================] - 2s 120ms/step - loss: 4.4788 - acc: 0.7191 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 125/150
20/20 [==============================] - 2s 114ms/step - loss: 4.5369 - acc: 0.7154 - val_loss: 2.7726 - val_acc: 0.8261
Epoch 126/150
20/20 [==============================] - 2s 113ms/step 

In [26]:
image = cv2.imread('/home/jailsonpereira/PAIC_CNN/teste.jpg',0)
output = image.copy()
image = cv2.resize(image, (150, 150))


image = image.astype("float32") / 255.0
print(image.shape)

image = np.array(image,dtype='float32')/255
image = image.reshape(-1,image.shape[0], image.shape[1],1)
print(image.shape)

pred = model.predict(image)

(150, 150)
(1, 150, 150, 1)


In [27]:
i = pred.argmax(axis=1)[0]

In [28]:
label = lb.classes_[i]

In [29]:
pred

array([[1.]], dtype=float32)

In [30]:
text = "{}: {:.2f}%".format(label, pred[0][i] * 100)
cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
    (0, 0, 255), 2)
 

array([[223, 222, 222, ..., 184, 184, 183],
       [221, 220, 220, ..., 184, 183, 183],
       [220, 220, 220, ..., 183, 183, 183],
       ...,
       [225, 225, 225, ..., 217, 217, 218],
       [225, 225, 225, ..., 216, 216, 217],
       [226, 226, 226, ..., 215, 216, 216]], dtype=uint8)

In [ ]:
cv2.imshow("Image", output)
cv2.waitKey(0)